In [ ]:
import zipfile, os, cv2
import numpy as np
import tensorflow as tf
tf.executing_eagerly()
from tensorflow import keras
# Display
from IPython.display import Image
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from keras import backend as K
K.set_image_data_format('channels_last')

In [ ]:
# np.save('np_multi/image_arr', image_arr)
# np.save('np_multi/mask_arr', mask_arr)
image_arr = np.load('np_multi/image_arr_cv1.npy').astype(np.float32)
mask_arr = np.load('np_multi/mask__arr_cv1.npy').astype(np.float32)
print(image_arr.shape, mask_arr.shape)
image_arr = np.transpose(image_arr, (0,3,1,2,4))
mask_arr = np.transpose(mask_arr, (0,3,1,2,4))
print(image_arr.shape, mask_arr.shape)

In [ ]:
x_train = image_arr[:60]
y_train = mask_arr[:60]
x_val = image_arr[60:]
y_val = mask_arr[60:]

In [ ]:
# 3d shape to 2d shape
print(x_train.shape, x_val.shape)
imgs_train = np.reshape(x_train,(x_train.shape[0]*x_train.shape[1],192,192,1))
imgs_mask_train = np.reshape(y_train,(y_train.shape[0]*y_train.shape[1],192,192,1))
imgs_valid = np.reshape(x_val,(x_val.shape[0]*x_val.shape[1],192,192,1))
imgs_mask_valid = np.reshape(y_val,(y_val.shape[0]*y_val.shape[1],192,192,1))
print(imgs_train.shape, imgs_valid.shape)
# image_arr = np.transpose(image_arr,(0,2,3,1))
# mask_arr = np.transpose(mask_arr,(0,2,3,1))

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
image_aug = ImageDataGenerator(
#             rotation_range = 20,
#             brightness_range=[0.5, 1.5],
            )

mask_aug = ImageDataGenerator(
#             rotation_range = 20,
#             brightness_range=[1, 1.5]
            )

In [ ]:
seed=1
batch_size=32
X_train_datagen = image_aug.flow(imgs_train, batch_size=batch_size, seed=seed)
Y_train_datagen = mask_aug.flow(imgs_mask_train, batch_size=batch_size, seed=seed)
train_generator = zip(X_train_datagen, Y_train_datagen)

X_valid_datagen = image_aug.flow(imgs_valid, batch_size=batch_size, seed=seed)
Y_valid_datagen = mask_aug.flow(imgs_mask_valid, batch_size=batch_size, seed=seed)
valid_generator = zip(X_valid_datagen, Y_valid_datagen)

In [ ]:
# num_batch = 32
# for i,batch in enumerate(X_valid_datagen):
#     if(i >= num_batch):
#         break

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)

def dice_p_bce(in_gt, in_pred):
    return 1e-3*binary_crossentropy(in_gt, in_pred) - dice_coef(in_gt, in_pred)

def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)

def focal_tversky_loss(y_true, y_pred, gamma=2.):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma)

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

In [ ]:
from model_2D_unet import *
# fitting shape [[slice, w, h, c], class]
model = get_unet()
# model.compile(optimizer=Adam(lr=1e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.000000199), 
#            loss='binary_crossentropy', metrics=['accuracy',dice_coef])
learning_rate = 1e-5
epoch = 30
learning_decay_rate = learning_rate/epoch
model.compile(optimizer=Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=None, 
                             decay=learning_decay_rate, amsgrad=False), 
           loss=dice_p_bce, metrics=['accuracy',dice_coef])

In [ ]:
from keras.callbacks import ModelCheckpoint, TensorBoard
weight_dir = 'checkpoint'
if not os.path.exists(weight_dir):
    os.mkdir(weight_dir)
# checkpoint_name = 'Unet_mri-epoch:{epoch:02d}-loss:{loss:.2f}-Dice:{dice_coef:.4f}.hdf5'
checkpoint_name = 'Unet_mri-best_cv1_noaug_u_2d'
model_checkpoint = ModelCheckpoint(os.path.join(weight_dir+'/metric_try2',f"{checkpoint_name}.hdf5"), 
                                   monitor='val_loss', mode="auto", verbose=0, save_best_only=True)
logdir = os.path.join("checkpoint/tensorboard/", checkpoint_name)
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1,
                                         embeddings_freq=0,embeddings_layer_names=None,)


In [ ]:
print('-'*30,'\nFitting model...\n','-'*30)
# history = model.fit(imgs_train, imgs_mask_train, batch_size=1, epochs=epoch, verbose=1, 
#                     shuffle=True, validation_data=(imgs_valid,imgs_mask_valid), 
#                     callbacks=[model_checkpoint,tensorboard_callback])

history = model.fit_generator(train_generator, epochs=epoch, verbose=1,
                    steps_per_epoch= len(imgs_train)//1,
                    shuffle=True, validation_data=valid_generator,
                    validation_steps= len(imgs_valid)//1,          
                    callbacks=[model_checkpoint,tensorboard_callback])


# history = model.fit(train_dataset, epochs=epoch, verbose=2, 
#                     shuffle=True, validation_data=validation_dataset, 
#                     callbacks=[model_checkpoint,tensorboard_callback])


In [ ]:
print('-'*30)
print('Loading and preprocessing test data...')
print('-'*30)


imgs_test = imgs_valid
imgs_mask_test = imgs_mask_valid


print('-'*30)
print('Loading saved weights...')
print('-'*30)

model = get_unet()
weight_dir = 'checkpoint/metric_try2'
checkpoint_name = checkpoint_name+'.hdf5'
model.load_weights(os.path.join(weight_dir,checkpoint_name ))

In [ ]:
Results = model.predict(imgs_test, batch_size=1, verbose=1)

In [ ]:
Results.shape

In [ ]:
import matplotlib.pyplot as plt
for j in range(Results.shape[0]):
        if np.max(imgs_test[j])>0:
            plt.figure(figsize=(10,4))
            plt.subplot(1,3,1)
            plt.imshow(np.squeeze(imgs_test[j]))
            plt.title('Original Image')
            plt.subplot(1,3,2)
            plt.imshow(np.squeeze(imgs_mask_test[j]))
            plt.title('Original Mask')
            plt.subplot(1,3,3)
            plt.imshow(np.squeeze(Results[j]) > .5)
            plt.title('Prediction')
            plt.show()